In [24]:
import sqlite3
import pandas as pd

In [25]:
con = sqlite3.connect(':memory:')

In [26]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [27]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [28]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [29]:
pd.read_sql(
    '''
SELECT CustomerName, Country, Address
FROM Customers
WHERE Country IN ('Germany', 'France') OR City = 'Madrid';
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [30]:
pd.read_sql(
    '''
SELECT Country, COUNT(*) AS CustomerCount
FROM Customers
GROUP BY Country
ORDER BY CustomerCount DESC
LIMIT 3;
    ''',
    con,
)

,Country,CustomerCount
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [31]:
pd.read_sql(
    '''
SELECT S.ShipperName, O.OrderDate
FROM Orders O
JOIN Shippers S ON O.ShipperID = S.ShipperID
ORDER BY O.OrderDate
LIMIT 1 OFFSET 9;
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [40]:
pd.read_sql(
    '''
SELECT P.ProductName, P.Price
FROM OrderDetails OD
JOIN Products P ON OD.ProductID = P.ProductID
WHERE OD.OrderID = (
    SELECT OD.OrderID
    FROM OrderDetails OD
    JOIN Products P ON OD.ProductID = P.ProductID
    GROUP BY OD.OrderID
    ORDER BY SUM(OD.Quantity * P.Price) DESC
    LIMIT 1
);
    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [33]:
pd.read_sql(
    '''
SELECT P.ProductName, SUM(OD.Quantity) AS TotalQuantity
FROM OrderDetails OD
JOIN Products P ON OD.ProductID = P.ProductID
GROUP BY P.ProductName
ORDER BY TotalQuantity DESC
LIMIT 1;
    ''',
    con,
)

,ProductName,TotalQuantity
0,Gorgonzola Telino,458


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [34]:
pd.read_sql(
    '''
SELECT S.SupplierName, S.Country, S.ContactName, S.Phone, COUNT(P.ProductID) AS OrderCount
FROM Suppliers S
JOIN Products P ON S.SupplierID = P.SupplierID
JOIN OrderDetails OD ON P.ProductID = OD.ProductID
GROUP BY S.SupplierID
ORDER BY OrderCount DESC
LIMIT 5;
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone,OrderCount
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,42
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,40
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448,33
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010,32
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323,31


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [35]:
pd.read_sql(
    '''
SELECT C.Country, CT.CategoryName, SUM(OD.Quantity * P.Price) AS TotalAmount
FROM Orders O
JOIN Customers C ON O.CustomerID = C.CustomerID
JOIN OrderDetails OD ON O.OrderID = OD.OrderID
JOIN Products P ON OD.ProductID = P.ProductID
JOIN Categories CT ON P.CategoryID = CT.CategoryID
WHERE C.Country = 'Brazil'
GROUP BY CT.CategoryName
ORDER BY TotalAmount DESC
LIMIT 1;
    ''',
    con,
)

,Country,CategoryName,TotalAmount
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [36]:
pd.read_sql(
    '''
WITH OrderAmounts AS (
    SELECT O.OrderID, SUM(OD.Quantity * P.Price) AS TotalAmount
    FROM Orders O
    JOIN Customers C ON O.CustomerID = C.CustomerID
    JOIN OrderDetails OD ON O.OrderID = OD.OrderID
    JOIN Products P ON OD.ProductID = P.ProductID
    WHERE C.Country = 'USA'
    GROUP BY O.OrderID
)
SELECT MAX(TotalAmount) - MIN(TotalAmount) AS PriceDifference
FROM OrderAmounts;
    ''',
    con,
)

,PriceDifference
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [37]:
pd.read_sql(
    '''
SELECT COUNT(O.OrderID) AS OrderCount, E.FirstName || ' ' || E.LastName AS FullName
FROM Orders O
JOIN Employees E ON O.EmployeeID = E.EmployeeID
GROUP BY E.EmployeeID
ORDER BY E.BirthDate DESC
LIMIT 3;
    ''',
    con,
)

,OrderCount,FullName
0,31,Janet Leverling
1,14,Robert King
2,40,Margaret Peacock


10. Сколько банок крабового мяса всего было заказано.

In [38]:
pd.read_sql(
    '''
SELECT SUM(OD.Quantity) AS TotalCans
FROM OrderDetails OD
JOIN Products P ON OD.ProductID = P.ProductID
WHERE P.ProductName = 'Crab Meat';
    ''',
    con,
)

,TotalCans
0,None
